In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import LSTM, Dense, Reshape

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [ ]:
#Load the already prepared data
aggLSTM = pd.read_parquet("Data/agg_table_new_zones8Bot.parquet")
#Create a pivot table
pivot_df = aggLSTM.pivot(index='time_bin', columns='h3_index', values='order_count').fillna(0)
#Create adjacency matrix on neighbours
adj_matrix = pd.read_parquet("Data/neighbours_zones_8.parquet")
adj_matrix.index = adj_matrix.columns

In [ ]:
#Scaling for normalisation
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(pivot_df)

In [ ]:
#Create sliding window for four steps
def create_dataset_4d(data, time_steps=4):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:i+time_steps, :].reshape((time_steps, data.shape[1], 1)))
        y.append(data[i+time_steps, :])
    return np.array(X), np.array(y)

SEQ_LEN = 4
X, y = create_dataset_4d(scaled_data, time_steps=SEQ_LEN)


In [ ]:
#One day test set
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y, test_size=0.125, shuffle=False
)

In [ ]:
# ~ one day validation set
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.15, shuffle=False
)

In [ ]:
X = X[..., np.newaxis]  # shape for training

In [ ]:
adj = adj_matrix
adj = adj + np.eye(adj.shape[0])  # Self-loop added

def normalize_adj(adj): #normalize adjacency feature
    D_inv_sqrt = np.diag(1.0 / np.sqrt(adj.sum(axis=1)))
    return D_inv_sqrt @ adj @ D_inv_sqrt

adj_normalized = normalize_adj(adj).astype(np.float32)
adj_tensor = tf.constant(adj_matrix, dtype=tf.float32)

In [ ]:
class GraphConvolution(tf.keras.layers.Layer):#Create GCN class for LSTM-GCN combination
    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.output_dim = output_dim

    def build(self, input_shape):
        self.kernel = self.add_weight(name='kernel',
                                      shape=(input_shape[-1], self.output_dim),
                                      initializer='glorot_uniform',
                                      trainable=True)

    def call(self, inputs, adj):
        x = tf.matmul(adj, inputs)  # (batch, nodes, features)
        x = tf.matmul(x, self.kernel)
        return x


class TimeDistributedGCN(tf.keras.layers.Layer):
    def __init__(self, output_dim, adj_matrix, **kwargs):
        super().__init__(**kwargs)
        self.output_dim = output_dim
        self.adj_matrix = adj_matrix
        self.gcn = GraphConvolution(output_dim)

    def call(self, inputs):  # inputs: (batch, time, nodes, features)
        outputs = []
        for t in range(inputs.shape[1]):
            x_t = inputs[:, t, :, :]  # (batch, nodes, features)
            out_t = self.gcn(x_t, self.adj_matrix)  # (batch, nodes, output_dim)
            outputs.append(out_t)
        return tf.stack(outputs, axis=1)  # (batch, time, nodes, output_dim)


In [ ]:
#Combining gcn with lstm
def build_gcn_lstm_model(num_nodes, seq_len, feature_dim, adj_matrix, gcn_units, lstm_units):
    X_input = Input(shape=(seq_len, num_nodes, feature_dim))
    
    gcn_td = TimeDistributedGCN(gcn_units, adj_matrix)
    gcn_output = gcn_td(X_input)  # (batch, time, nodes, gcn_units)
    
    reshaped = Reshape((seq_len, num_nodes * gcn_units))(gcn_output)
    lstm_out = LSTM(lstm_units, return_sequences=False)(reshaped)
    
    output = Dense(num_nodes)(lstm_out)

    return Model(inputs=X_input, outputs=output)


#Use model for training
model = build_gcn_lstm_model(
    num_nodes=X.shape[2],
    seq_len=X.shape[1],
    feature_dim=1,
    adj_matrix=adj_tensor,
    gcn_units=16,
    lstm_units=64
)
def log_cosh_loss(y_true, y_pred):
    return tf.reduce_mean(tf.math.log(tf.cosh(y_pred - y_true)))

model.compile(optimizer='adam', loss=log_cosh_loss)#Calculate training losses

model.summary()

In [ ]:
#Training
print("Begin Training...")
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

In [ ]:
#Plot with validation loss
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
#Prediction and rescale the values
predictions = model.predict(X_test)
predictions_rescaled = scaler.inverse_transform(predictions)
y_test_rescaled = scaler.inverse_transform(y_test)

In [ ]:
# Merge all values for metrics calculation and visualizaion
y_tests_all = np.concatenate(y_test_rescaled)
y_preds_all = np.concatenate(predictions_rescaled)
n = len(y_tests_all)
p = X.shape[1]

#all necessary metrics
rmse = np.sqrt(mean_squared_error(y_tests_all, y_preds_all))
mae = mean_absolute_error(y_tests_all, y_preds_all)
r2 = r2_score(y_tests_all, y_preds_all)
nrmse = rmse / np.mean(np.abs(y_tests_all)) * 100
nonzero_indices = y_tests_all != 0
adj_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)

#To not divide by zero
if np.any(nonzero_indices):
    mape = np.mean(np.abs((y_tests_all[nonzero_indices] - y_preds_all[nonzero_indices]) / y_tests_all[nonzero_indices])) * 100
else:
    mape = np.nan

print("Evaluation über alle Areas hinweg:")
print(f"MAPE: {mape:.2f}%")
print(f"NRMSE: {nrmse:.2f}%")
print(f"RMSE: {rmse:.2f}")
print(f"MAE: {mae:.2f}")
print(f"R²: {r2:.2f}")
print(f"Adjusted R²: {adj_r2:.2f}")

In [ ]:
example_area = "88329b5aa1fffff"  # The area to compare
example_index = pivot_df.columns.get_loc(example_area)
np.save('prognose_GCN_LSTM.npy', predictions_rescaled[:, example_index]) #Save output numpy file


plt.figure(figsize=(14, 6))
plt.plot(y_test_rescaled[:, example_index], label='Actual')
plt.plot(predictions_rescaled[:, example_index], label='Prediction')
plt.title(f'LSTM Prediction {example_area}')
plt.xlabel('Test-Time bins')
plt.ylabel('Orders')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Metrics for all comnined predictions total actual vs. total predicted
total_actual = y_test_rescaled.sum(axis=1)
total_pred = predictions_rescaled.sum(axis=1)

rmse = np.sqrt(mean_squared_error(total_actual, total_pred))
mae = mean_absolute_error(total_actual, total_pred)
r2 = r2_score(total_actual, total_pred)
mape = np.mean(np.abs((total_actual - total_pred) / np.maximum(total_actual, 1e-5))) * 100

print("\n--- Evaluation ---")
print(f"RMSE: {rmse:.2f}")
print(f"MAE:  {mae:.2f}")
print(f"R²:   {r2:.2f}")
print(f"MAPE: {mape:.2f}%")

In [ ]:
# Plot the cumulated demand over time
plt.figure(figsize=(12, 6))
plt.plot(total_actual, label="Actual Total Demand")
plt.plot(total_pred, label="Predicted Total Demand")
plt.title("Demand")
plt.xlabel("Timestep")
plt.ylabel("Orders")
plt.legend()
plt.grid(True)
plt.show()